In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

import duckdb
def query(s):
    pd.options.display.max_rows = 50
    return duckdb.sql(s).df()

In [ ]:
# Download and save the file locally 
books_metadata = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_metadata_sample_20k.csv",
)

books_reviews = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_reviews_sample_20k.csv",
)

# (Optional) Save the file locally 
import os

outdir = './dataset'
if not os.path.exists(outdir):
    os.mkdir(outdir)

books_metadata.to_csv('./dataset/metadata.csv',)
books_reviews.to_csv('./dataset/reviews.csv')


In [4]:
books_metadata = pd.read_csv('./dataset/metadata.csv', index_col=False)
books_reviews = pd.read_csv('./dataset/reviews.csv', index_col=False)


In [7]:
books_metadata.head()

,title,subtitle,author_name,author_about,publisher,publisher_date,format,page_count,language,isbn_10,...,price_numeric,description,features_text,dimensions,item_weight,images,videos,store,parent_asin,bought_together
0,Government Bullies: How Everyday Americans are...,"Paperback – August 6, 2013",Rand Paul,"Discover more of the author’s books, see simil...",Center Street,"41260th edition (August 6, 2013",Paperback,320.0,English,1455522775,...,12.99,"['Review', 'Praise for The Tea Party Goes to W...",Government regulations are out of control. The...,5.25 x 0.81 x 8 inches,9.2 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"Rand Paul (Author), Ron Paul (Foreword)",1455522775,NaN
1,Gordon Pope Thrillers: Books 1-3,"Paperback – December 17, 2020",B. B. Griffith,"B. B. Griffith was born and raised in Denver, ...",Griffith Publishing LLC,"December 17, 2020",Paperback,694.0,English,1735305855,...,24.69,"['Review', 'Selected Reviews from the Gordon P...",Gordon Pope has a special talent. | He's the o...,6 x 1.74 x 9 inches,2.21 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],B. B. Griffith (Author),1735305855,NaN
2,Chart Sense: Common Sense Charts to Teach 3-8 ...,"Paperback – Illustrated, February 2, 2014",Rozlyn Linder,Follow Roz on Twitter (@rozlinder) and connect...,"Literacy Initiative, The","Illustrated edition (February 2, 2014",Paperback,182.0,English,0988950510,...,22.88,"['Review', ""This is the first book I have ever...",Chart Sense is the ultimate resource for eleme...,7.5 x 0.41 x 9.25 inches,12 ounces,[],[],Rozlyn Linder (Author),0988950510,NaN
3,Where the Sky Begins: A Novel,"Paperback – August 2, 2022",Rhys Bowen,Rhys Bowen is the New York Times bestselling a...,Lake Union Publishing,"August 2, 2022",Paperback,399.0,English,1542028841,...,6.15,"['Review', '“Bowen is a master at winning read...",A woman’s future is determined by fate and cho...,5.5 x 1 x 8.25 inches,14.1 ounces,[],[],Rhys Bowen (Author),1542028841,NaN
4,The Mystery of Garabandal: Fantasy or Fraud? G...,NaN,LR Walker,"Discover more of the author’s books, see simil...",Newtonia Publishing LLC,"First Paperback Edition (July 31, 2015",Paperback,299.0,English,0692297677,...,12.99,"['About the Author', ""LR Walker studied journa...",This 2021 update includes new interviews with ...,6 x 0.68 x 9 inches,14.3 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"LR Walker (Author), Frank Lafleur (Introduction)",0692297677,NaN


In [5]:
pd.options.display.max_rows = 10
pd.options.display.min_rows = 10

# query("select right(publisher_date, 4) as year, count(distinct title) from books_metadata group by 1")
# query("select distinct right(publisher_date, 4) as year from books_metadata order by year")
processed_metadata = query("""
      
    with p1 as (
      select 
        * exclude(publisher_date),
        case 
          when try_cast(right(substr(publisher_date, strpos(publisher_date, '(')),4) as integer) is not null then split(publisher_date,'(')[-1]
          else null
        end as date_str,
      from books_metadata 
      -- where parent_asin in ('0962272809')
    )
    
    select 
      *,
      case when length(date_str) > 8 then strptime(date_str, '%B %-d, %Y') else null end published_date
    from p1
    """)

# The publisher date is like a scraping result from a dumb condition so we have to process our metadata to get a proper published_date column

processed_metadata

,Unnamed: 0,title,subtitle,author_name,author_about,publisher,format,page_count,language,isbn_10,...,features_text,dimensions,item_weight,images,videos,store,parent_asin,bought_together,date_str,published_date
0,0,Government Bullies: How Everyday Americans are...,"Paperback – August 6, 2013",Rand Paul,"Discover more of the author’s books, see simil...",Center Street,Paperback,320.0,English,1455522775,...,Government regulations are out of control. The...,5.25 x 0.81 x 8 inches,9.2 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"Rand Paul (Author), Ron Paul (Foreword)",1455522775,NaN,"August 6, 2013",2013-08-06
1,1,Gordon Pope Thrillers: Books 1-3,"Paperback – December 17, 2020",B. B. Griffith,"B. B. Griffith was born and raised in Denver, ...",Griffith Publishing LLC,Paperback,694.0,English,1735305855,...,Gordon Pope has a special talent. | He's the o...,6 x 1.74 x 9 inches,2.21 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],B. B. Griffith (Author),1735305855,NaN,"December 17, 2020",2020-12-17
2,2,Chart Sense: Common Sense Charts to Teach 3-8 ...,"Paperback – Illustrated, February 2, 2014",Rozlyn Linder,Follow Roz on Twitter (@rozlinder) and connect...,"Literacy Initiative, The",Paperback,182.0,English,0988950510,...,Chart Sense is the ultimate resource for eleme...,7.5 x 0.41 x 9.25 inches,12 ounces,[],[],Rozlyn Linder (Author),0988950510,NaN,"February 2, 2014",2014-02-02
3,3,Where the Sky Begins: A Novel,"Paperback – August 2, 2022",Rhys Bowen,Rhys Bowen is the New York Times bestselling a...,Lake Union Publishing,Paperback,399.0,English,1542028841,...,A woman’s future is determined by fate and cho...,5.5 x 1 x 8.25 inches,14.1 ounces,[],[],Rhys Bowen (Author),1542028841,NaN,"August 2, 2022",2022-08-02
4,4,The Mystery of Garabandal: Fantasy or Fraud? G...,None,LR Walker,"Discover more of the author’s books, see simil...",Newtonia Publishing LLC,Paperback,299.0,English,0692297677,...,This 2021 update includes new interviews with ...,6 x 0.68 x 9 inches,14.3 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"LR Walker (Author), Frank Lafleur (Introduction)",0692297677,NaN,"July 31, 2015",2015-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,The Prayer of the Lord,"Hardcover – May 26, 2009",R. C. Sproul,Dr. R.C. Sproul (1939–2017) was founder of Lig...,Reformation Trust Publishing,Hardcover,130.0,English,1567691188,...,What is the Lord’s Prayer? In | The Prayer of ...,5.31 x 0.63 x 7.75 inches,9.6 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],R.C. Sproul (Author),1567691188,NaN,"May 26, 2009",2009-05-26
19996,19996,A Christmas Carol,"Hardcover – September 3, 2012",Robert Ingpen,"Robert Ingpen was born in 1936 in Geelong, Aus...",Chartwell Books,Hardcover,128.0,English,0785829466,...,Dickens was always fond of | A Christmas Carol...,9.63 x 0.5 x 11.75 inches,1.69 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],"Charles Dickens (Author), Arthur Rackham (Ill...",0785829466,NaN,"September 3, 2012",2012-09-03
19997,19997,An Irish Christmas,None,Melody Carlson,Melody Carlson has written more than 200 books...,Revell,Hardcover,192.0,English,None,...,"For Colleen, life is spinning out of control. ...",8.76 x 5.84 x 0.72 inches,8 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],Melody Carlson (Author),B002SB8NQ2,NaN,"September 1, 2007",2007-09-01
19998,19998,Darcy and Elizabeth: Hope of the Future (Darcy...,"Paperback – August 10, 2017",Sharon Lathan,SHARON LATHAN is the best-selling author of TH...,Sharon Lathan,Paperback,370.0,English,0991610628,...,Fitzwilliam Darcy and Elizabeth Bennet will so...,6 x 0.93 x 9 inches,1.19 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],"Sharon Lathan (Author), Gretchen Stelter (Edi...",0991610628,NaN,"August 10, 2017",2017-08-10


In [7]:
# set pandas max column to 50 
pd.options.display.max_columns = 50

processed_metadata.head()

,Unnamed: 0,title,subtitle,author_name,author_about,publisher,format,page_count,language,isbn_10,isbn_13,main_category,category_level_1_main,category_level_2_sub,category_level_3_detail,average_rating,rating_number,price,price_numeric,description,features_text,dimensions,item_weight,images,videos,store,parent_asin,bought_together,date_str,published_date
0,0,Government Bullies: How Everyday Americans are...,"Paperback – August 6, 2013",Rand Paul,"Discover more of the author’s books, see simil...",Center Street,Paperback,320.0,English,1455522775,978-1455522774,Books,Books,Politics & Social Sciences,Politics & Government,4.6,274,12.99,12.99,"['Review', 'Praise for The Tea Party Goes to W...",Government regulations are out of control. The...,5.25 x 0.81 x 8 inches,9.2 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"Rand Paul (Author), Ron Paul (Foreword)",1455522775,NaN,"August 6, 2013",2013-08-06
1,1,Gordon Pope Thrillers: Books 1-3,"Paperback – December 17, 2020",B. B. Griffith,"B. B. Griffith was born and raised in Denver, ...",Griffith Publishing LLC,Paperback,694.0,English,1735305855,978-1735305851,Books,Books,Literature & Fiction,Genre Fiction,4.6,626,24.69,24.69,"['Review', 'Selected Reviews from the Gordon P...",Gordon Pope has a special talent. | He's the o...,6 x 1.74 x 9 inches,2.21 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],B. B. Griffith (Author),1735305855,NaN,"December 17, 2020",2020-12-17
2,2,Chart Sense: Common Sense Charts to Teach 3-8 ...,"Paperback – Illustrated, February 2, 2014",Rozlyn Linder,Follow Roz on Twitter (@rozlinder) and connect...,"Literacy Initiative, The",Paperback,182.0,English,0988950510,978-0988950511,Books,Books,Education & Teaching,Schools & Teaching,4.7,272,22.88,22.88,"['Review', ""This is the first book I have ever...",Chart Sense is the ultimate resource for eleme...,7.5 x 0.41 x 9.25 inches,12 ounces,[],[],Rozlyn Linder (Author),0988950510,NaN,"February 2, 2014",2014-02-02
3,3,Where the Sky Begins: A Novel,"Paperback – August 2, 2022",Rhys Bowen,Rhys Bowen is the New York Times bestselling a...,Lake Union Publishing,Paperback,399.0,English,1542028841,978-1542028844,Books,Books,Literature & Fiction,Genre Fiction,4.5,27244,6.15,6.15,"['Review', '“Bowen is a master at winning read...",A woman’s future is determined by fate and cho...,5.5 x 1 x 8.25 inches,14.1 ounces,[],[],Rhys Bowen (Author),1542028841,NaN,"August 2, 2022",2022-08-02
4,4,The Mystery of Garabandal: Fantasy or Fraud? G...,None,LR Walker,"Discover more of the author’s books, see simil...",Newtonia Publishing LLC,Paperback,299.0,English,0692297677,978-0692297674,Books,Books,Christian Books & Bibles,Theology,4.5,287,12.99,12.99,"['About the Author', ""LR Walker studied journa...",This 2021 update includes new interviews with ...,6 x 0.68 x 9 inches,14.3 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"LR Walker (Author), Frank Lafleur (Introduction)",0692297677,NaN,"July 31, 2015",2015-07-31


In [ ]:
pd.options.display.max_rows = 30
pd.options.display.min_rows = 30


query(
    """
    with p1 as (
    select 
        publisher, count(distinct paernt_asin)
        -- title, published_date, author_name, total_review
    from processed_metadata a

    group by 1
    order by 2 desc)

    select author_name, sum(total_review) from p1 group by 1 order by 2 desc
"""
)

,author_name,sum(total_review)
0,John Grisham,26436.0
1,Michael Connelly,7939.0
2,James Patterson,7212.0
3,Suzanne Collins,5073.0
4,Deborah Harkness,4492.0
5,Jim Butcher,4014.0
6,Stephen King,3693.0
7,Robert Galbraith,3437.0
8,Jeffrey Eisner,2848.0
9,Jeff Kinney,2766.0


In [ ]:
query("""
    select 
        year(m.published_date) as year,
        m.category_level_3_detail as genre,
        count(distinct m.parent_asin) as book_count,
        count(r.asin) as review_count,
        sum(r.rating * m.price) as total_sales
    from processed_metadata m
    left join books_reviews r using(parent_asin)
    where m.published_date is not null and m.category_level_3_detail is not null
    group by year(m.published_date), m.category_level_3_detail
    order by year, book_count desc
""")

,main_category,format,total_books
0,Buy a Kindle,None,1237
1,Books,Paperback,9901
2,Books,None,3219
3,Books,Hardcover,5643


In [ ]:
books_reviews.head()

,Unnamed: 0,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,date,year
0,0,5.0,Good choice,Just what I was looking for. Thank you.,[],1477493395,1477493395,AGXFEGMNVCSTSYYA5UWXDV7AFSXA,1529789386700,1,True,2018-06-23 21:29:46.700,2018
1,1,3.0,Perfect size,Very small lines to color had to get 4.0 marke...,[],1780679548,1780679548,AGPR2UDFIJJCBA3EKFEEAJBWKROQ,1591370837527,0,True,2020-06-05 15:27:17.527,2020
2,2,1.0,Much of this book was unbelievable and I almos...,Hard to get through was very disappointed with...,[],1785588451,1785588451,AEUH4EH6XHROLT7UZPUYU2YKTYMA,1618447014407,1,True,2021-04-15 00:36:54.407,2021
3,3,5.0,Great Buy!,"This series was requested by my grandson, who ...",[],1606412388,1606412388,AH6PLOGWYIVIWLJTY756BHNFD4YA,1340195874000,0,True,2012-06-20 12:37:54.000,2012
4,4,5.0,You can go home again!,"I love John Grogan, he has always been one of ...",[],0061713244,0061713244,AFSKPY37N3C43SOI5IEXEK5JSIYA,1224796701000,1,False,2008-10-23 21:18:21.000,2008


In [5]:
samplebook = query(
    """    
    select
        title, 
        publisher_date, 
        author_name,
        cast(trim(right(publisher_date,4)) as int64) published_year, 
        parent_asin 
    from books_metadata 
    where 1=1 
        and lower(author_name) like '%suzanne collins%'
    order by published_year desc
        limit 100
    """
)
p1 = query(
    """
select
      *
from books_reviews
where parent_asin in (select distinct parent_asin from samplebook)
-- limit 5      
"""
)

query("""
select parent_asin, count(1) from p1 group by 1 order by 2 desc
      """)

,parent_asin,count(1)
0,0439023521,1
1,2266182714,1
2,0545265355,1
3,1338321919,1
4,043979143X,1
5,0545422906,1


In [ ]:
select * from books_metadata
where lower()

,parent_asin,count(1)
0,0439023521,3150
1,0545265355,1827
2,0545422906,77
3,043979143X,17
4,2266182714,1
5,1338321919,1
